In [1]:
import sys
sys.path.append('../')

import matlab.engine
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skopt.sampler import Lhs
from skopt.space import Space
import math
from pyswarms.single.global_best import GlobalBestPSO
import pyswarms as ps
from scipy.stats import norm
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)
from sklearn.neighbors import KernelDensity
from numpy.random import uniform
from sklearn.preprocessing import MinMaxScaler
import matplotlib.transforms as transforms
from matplotlib.patches import Ellipse
from Hypervolume import HyperVolume
import optproblems.zdt
import ConfigSpace as CS
import sys
import matplotlib.pyplot as plt

from Problem import Problem
from MOTPE import MOTPE
# Problems
from DTLZ import DTLZ
from WFG import WFG
from ZDT import ZDT1

eng = matlab.engine.start_matlab()
random_seed = 1990

In [2]:
def compute_hypervolume(m, referencePoint):
    hv = HyperVolume(referencePoint)
    volume = hv.compute(m)
    return volume

In [3]:
def pareto_front(points, level=0, index=False):

    points = points.copy()
    ranks = np.zeros(len(points))
    r = 0
    c = len(points)
    while c > 0:
        extended = np.tile(points, (points.shape[0], 1, 1))
        dominance = np.sum(np.logical_and(
            np.all(extended <= np.swapaxes(extended, 0, 1), axis=2),
            np.any(extended < np.swapaxes(extended, 0, 1), axis=2)), axis=1)
        points[dominance == 0] = 1e9  # mark as used
        ranks[dominance == 0] = r
        r += 1
        c -= np.sum(dominance == 0)
    if index:
#         return ranks
        return [i for i in range(len(ranks)) if ranks[i] == level] 
    else:    
        ind = [i for i in range(len(ranks)) if ranks[i] == level]
        return points[ind]

In [4]:
def acquisition_function(candidates, model, Z_min, score_tpe=0):
    BK = np.asarray([np.ones(candidates.shape[0])])
    BK = matlab.double(np.transpose(BK).tolist())

    [SK_gau, det_mse]  = eng.SKpredict(model, matlab.double(candidates.tolist()), BK, 
                                              matlab.double(np.asarray([random_seed]).tolist()),
                                              nargout=2)
    SK_gau = np.asarray(SK_gau)
    det_mse = np.asarray(det_mse)
    mse = np.sqrt(np.abs(det_mse))
    
    mei = (Z_min-SK_gau) * norm.cdf((Z_min-SK_gau)/mse,0,1) + mse * norm.pdf((Z_min-SK_gau)/mse,0,1) + score_tpe
#     print("mei", mei, mei.shape)
    return -mei[:,0]

In [5]:
eps = sys.float_info.epsilon

ideal_point = np.asarray([0, 0])
problem_names = ["ZDT1", "WFG4", "DTLZ7"]

lambdas = np.linspace(0.01, 0.1, 10)
gammas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

for id_problem in range(len(problem_names)):

    for l in lambdas:
        for g in gammas:
            seed                = random_seed   #args.seed
            num_variables       = 5     #args.num_variables
            num_initial_samples = 11*num_variables-1    #args.num_initial_samples
            num_max_evals       = 100   #args.num_max_evals
            num_objectives      = 2     #args.num_objectives
            replications        = 50

            k                   = 1     #args.k
            num_candidates      = 1000    #args.num_candidates
            init_method         = 'lhs' #args.init_method
            gamma               = g   #args.gamma
            base_configuration  = {
                'num_objectives': num_objectives,
                'num_variables': num_variables,
                'replications':replications,
                'k': k,
                'seed': seed}
            benchmark_name      = problem_names[id_problem] #args.benchmark_name

            if benchmark_name == "ZDT1":
                f       = ZDT1(benchmark_name, base_configuration)
                range_objectives = np.asarray([[1, 10],
                                  [0, 0]])
                hv_point = [1, 10]
            elif benchmark_name == "WFG4":
                f       = WFG(benchmark_name, base_configuration)
                range_objectives = np.asarray([[3, 5],
                                          [0, 0]])
                hv_point = [3, 5]
            else: #DTLZ7
                f       = DTLZ(benchmark_name, base_configuration)
                range_objectives = np.asarray([[1, 23],
                                  [0, 0]])
                hv_point = [1, 23]

            cs      = f.make_cs(CS.ConfigurationSpace(seed=seed))
            problem = Problem(f, cs)
            solver  = MOTPE(seed=seed)

            constants = solver.prepare_heterogeneus_noise(range_objectives, rep_budget=replications)
            cs, i = solver.init_model(
                {'num_initial_samples': num_initial_samples,
                 'num_max_evals': num_max_evals,
                 'init_method': init_method,
                 'num_candidates': num_candidates,
                 'scalarize':False,
                 'gamma': gamma,
                 'newSampling': l},
                    problem)
            # data = solver.history_replications()
            # X = solver.get_X()
            hv_history = []
            W = solver.get_weights_scalarization()

            try:
                #Optimization procedure
                for it in range(num_max_evals):   
                    print("Iteration: ", it, "/", num_max_evals)

                    data = solver.history_replications()[:,:2] #Because the last column has the iteration number
                    X = solver.get_X()

                    #Prepare optimization
                    Y_mean=np.zeros(X.shape[0])
                    Vhat=np.zeros(X.shape[0])
                    B = np.asarray([np.ones(X.shape[0])])

                    #### Scalarization
                    # Randomly select a point
                    weight = W[np.random.randint(0, W.shape[0]),:]

                    for i in range(0, int(data.shape[0]/replications), replications):
                        point = np.asarray(data[i:i+replications, :])
                        
                        ip_replicated = np.tile(ideal_point, (replications,1))
                        
                        pcheby_term1 = np.max((point - ip_replicated) * np.tile(weight, (replications,1)), axis=1)
                        pcheby_term2 = np.sum((point - ip_replicated) * np.tile(weight, (replications,1)), axis=1)
                        pcheby = pcheby_term1 + 0.05*pcheby_term2
                        Y_mean[i] = np.mean(pcheby);
                        Vhat[i] = np.var(pcheby)/replications; #because var already considers the division by N
                    Vhat = np.asarray([Vhat]).T
                    #Normalize objectives
                    scaler = MinMaxScaler()
                    Y_mean = scaler.fit_transform(np.asarray([Y_mean]).T)


#                     print(X.shape, Y_mean.shape, B.T.shape, Vhat.shape)

                    X_matlab = matlab.double(X.tolist())
                    Y_matlab = matlab.double(Y_mean.tolist())
                    B = matlab.double(np.transpose(B).tolist())
                    Vhat = matlab.double(Vhat.tolist())

#                     skriging_model_2 = eng.SKfit_new(X_matlab, Y_matlab, B, Vhat, 2, 3, 
#                                                      matlab.double(np.asarray([random_seed]).tolist()), nargout=1);
                    skriging_model_2 = eng.SKfit(X_matlab, Y_matlab, B, Vhat, 2,  
                                                     matlab.double(np.asarray([random_seed]).tolist()), nargout=1);

                #     ######### SELECTION ########
                    # Select the best point and evaluate it wh SK
                    index_best = Y_mean.argmin(axis=0)[0]
                    # Select the point with the best scalarized value
                    X_matlab = matlab.double(np.asarray(X[index_best,:]).tolist())
                    #Evaluate the point with the metamodel
                    [SK_gau, det_mse]  = eng.SKpredict(skriging_model_2, X_matlab, 
                                                              matlab.double(np.asarray([1]).tolist()), 
                                                              matlab.double(np.asarray([random_seed]).tolist()),
                                                              nargout=2);
                    SK_gau_array = np.asarray(SK_gau)
                    sample_tpe = solver.sample(cs)
                    #Since I am using the new sampling, the last configurations have the maximum score given by MOTPE. 
                    # Therefore, the final configuration will be selected from gamma*sample
                    sample = sample_tpe[int((1-l)*sample_tpe.shape[0]):,:] #Use only the last candidate defined by lambda

                    # Select the point sampled from l that maximize the MEI
                    meis = acquisition_function(sample, skriging_model_2, SK_gau_array, l)
                    best_mei = meis.argmin(axis=0)
                    pos = sample[best_mei]
                    solver.add_infill_point(problem, pos, i) #Evaluate the point
                    data = solver.history_replications() #To retrieve the evaluation of the new point
                    # Compute the means to obtan the Pareto front
                    unique = np.unique(data[:,2], return_counts =False)
                    m = np.asarray([np.mean(data[data[:,2]==di, :2], axis=0) for di in unique])
                    
                    pf = pareto_front(m, index=True)
                    pf = m[pf]
                    # Compute the hypervolume
                    volume=compute_hypervolume(pf, hv_point)
                    hv_history.append(volume)
                #     plot_evolution(X, XK, skriging_model_2, truthk, it)

                # Save the replications
                rep = solver.history_replications()
                mmm = pd.DataFrame(rep) #rep.reshape(rep.shape[0]*rep.shape[1],2)
                mmm.to_csv("PF_data/replications_GP_MOTPE_"+benchmark_name+"_gamma_"+str(g)+"_l_"+str(l)+".csv")
                # Save the exploration
                m = solver.get_F()
                df = pd.DataFrame(m)
                df.to_csv("PF_data/exploration_GP_MOTPE_"+benchmark_name+"_gamma_"+str(g)+"_l_"+str(l)+".csv")
            except:
                print("MATLAB execution error")
                print("Save *********************************!!!")
                # Save the replications
                rep = solver.history_replications()
                mmm = pd.DataFrame(rep) #rep.reshape(rep.shape[0]*rep.shape[1],2)
                mmm.to_csv("PF_data/replications_GP_MOTPE_"+benchmark_name+"_gamma_"+str(g)+"_l_"+str(l)+".csv")
                # Save the exploration
                m = solver.get_F()
                df = pd.DataFrame(m)
                df.to_csv("PF_data/exploration_GP_MOTPE_"+benchmark_name+"_gamma_"+str(g)+"_l_"+str(l)+".csv")

Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
Iteration:  16 / 100
Iteration:  17 / 100
Iteration:  18 / 100
Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
Iteration:  24 / 100
Iteration:  25 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
Iteration:  16 / 100


Iteration:  16 / 100
Iteration:  17 / 100
Iteration:  18 / 100
Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
Iteration:  24 / 100
Iteration:  25 / 100
Iteration:  26 / 100
Iteration:  27 / 100
Iteration:  28 / 100
Iteration:  29 / 100
Iteration:  30 / 100
Iteration:  31 / 100
Iteration:  32 / 100
Iteration:  33 / 100
Iteration:  34 / 100
Iteration:  35 / 100
Iteration:  36 / 100
Iteration:  37 / 100
Iteration:  38 / 100
Iteration:  39 / 100
Iteration:  40 / 100
Iteration:  41 / 100
Iteration:  42 / 100
Iteration:  43 / 100
Iteration:  44 / 100
Iteration:  45 / 100
Iteration:  46 / 100
Iteration:  47 / 100
Iteration:  48 / 100
Iteration:  49 / 100
Iteration:  50 / 100
Iteration:  51 / 100
Iteration:  52 / 100
Iteration:  53 / 100
Iteration:  54 / 100
Iteration:  55 / 100
Iteration:  56 / 100
Iteration:  57 / 100
Iteration:  58 / 100
Iteration:  59 / 100
Iteration:  60 / 100
Iteration:  61 / 100
Iteration:  62 / 100
Iteration:  6

Iteration:  16 / 100
Iteration:  17 / 100
Iteration:  18 / 100
Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
Iteration:  16 / 100
Iteration:  17 / 100
Iteration:  18 / 100
Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
Iteration:  24 / 100
Iteration:  25 / 100
Iteration:  26 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4

Iteration:  74 / 100
Iteration:  75 / 100
Iteration:  76 / 100
Iteration:  77 / 100
Iteration:  78 / 100
Iteration:  79 / 100
Iteration:  80 / 100
Iteration:  81 / 100
Iteration:  82 / 100
Iteration:  83 / 100
Iteration:  84 / 100
Iteration:  85 / 100
Iteration:  86 / 100
Iteration:  87 / 100
Iteration:  88 / 100
Iteration:  89 / 100
Iteration:  90 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
Iteration:  16 / 100
Iteration:  17 / 100
Iteration:  18 / 100
Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
Iteration:  24 / 100
Iteration:  25 / 100
Iteration:  26 /

Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
Iteration:  16 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
Iteration:  16 / 100
Iteration:  17 / 100
Iteration:  18 / 100
Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
Iteration:  24 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 

...

Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
Iteration:  16 / 100
Iteration:  17 / 100
Iteration:  18 / 100
Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
Iteration:  24 / 100
Iteration:  25 / 100
Iteration:  26 / 100
Iteration:  27 / 100
Iteration:  28 / 100
Iteration:  29 / 100
Iteration:  30 / 100
Iteration:  31 / 100
Iteration:  32 / 100
Iteration:  33 / 100
Iteration:  34 / 100
Iteration:  35 / 100
Iteration:  36 / 100
Iteration:  37 / 100
Iteration:  38 / 100
Iteration:  39 / 100
Iteration:  40 / 100
Iteration:  41 / 100
Iteration:  42 / 100
Iteration:  43 / 100
Iteration:  44 / 100
Iteration:  45 / 100
Iteration:  46 / 100
Iteration:  47 / 100
Iteration:  48 / 100
Iteration:  49 / 100
Iteration:  50 / 100
Iteration:  51 / 100
Iteration:  52 / 100
Iteration:  53 / 100
Iteration:  54 / 100
Iteration:  55 / 100
Iteration:  56 / 100
Iteration:  57 / 100
Iteration:  5

Iteration:  53 / 100
Iteration:  54 / 100
Iteration:  55 / 100
Iteration:  56 / 100
Iteration:  57 / 100
Iteration:  58 / 100
Iteration:  59 / 100
Iteration:  60 / 100
Iteration:  61 / 100
Iteration:  62 / 100
Iteration:  63 / 100
Iteration:  64 / 100
Iteration:  65 / 100
Iteration:  66 / 100
Iteration:  67 / 100
Iteration:  68 / 100
Iteration:  69 / 100
Iteration:  70 / 100
Iteration:  71 / 100
Iteration:  72 / 100
Iteration:  73 / 100
Iteration:  74 / 100
Iteration:  75 / 100
Iteration:  76 / 100
Iteration:  77 / 100
Iteration:  78 / 100
Iteration:  79 / 100
Iteration:  80 / 100
Iteration:  81 / 100
Iteration:  82 / 100
Iteration:  83 / 100
Iteration:  84 / 100
Iteration:  85 / 100
Iteration:  86 / 100
Iteration:  87 / 100
Iteration:  88 / 100
Iteration:  89 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  

Iteration:  9 / 100
Iteration:  10 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
Iteration:  14 / 100
Iteration:  15 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100

Iteration:  17 / 100
Iteration:  18 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
MATLAB execution err

Iteration:  19 / 100
Iteration:  20 / 100
Iteration:  21 / 100
Iteration:  22 / 100
Iteration:  23 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 100
Iteration:  8 / 100
Iteration:  9 / 100
Iteration:  10 / 100
Iteration:  11 / 100
Iteration:  12 / 100
Iteration:  13 / 100
MATLAB execution error
Save *********************************!!!
Sampling initial points: 
Iteration:  0 / 100
Iteration:  1 / 100
Iteration:  2 / 100
Iteration:  3 / 100
Iteration:  4 / 100
Iteration:  5 / 100
Iteration:  6 / 100
Iteration:  7 / 10